<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/main/TF_IDF_UP2.5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar la base

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
base=pd.read_csv('/content/drive/MyDrive/base_formateada.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Se incorporan las stopwords sugeridas por Federico
stopwords.extend(['señor', 'señora', 'mucha', 'gracia', 'año', 'pasado', 'cada', 'vez', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis' ,'siete', 'ocho', 'nueve', 'diez'])
stopwords.extend(['hoy', 'aquí', 'primer', 'lugar', 'primera', 'segunda', 'primero', 'segundo', 'siguiente', 'tercer'])
stopwords.extend(['convergencia', 'esquerra', 'republicana', 'grupo', 'parlamentaria', 'parlamentario', 'partido'])
stopwords.extend(['enmienda', 'votación', 'favor', 'abstención', 'ley', 'real', 'decreto', 'decretoley', 'resultado', 'voto', 'sé' ,'sí', 'silencio' ,'favor'])
stopwords.extend(['usted', 'señoría', 'presidente', 'presidenta', 'ministro', 'orden' , 'día', 'palabra', 'petición', 'posición', 'punto', 'vista', 'sesión', 'baldoví', 'duran', 'turno'])
stopwords.extend(['hacer', 'frente', 'puede' ,'ser', 'va', 'voy', 'decir'])
stopwords.extend(['millón', 'euro', 'emitido', 'efectuada', 'dio', 'comienzo', 'partido', 'queda', 'quedan', 'rechazada', 'aceptada', 'comienza', 'usted', 'sabe', 'abstención', 'diputado', 'gobierno'])
stopwords.extend(['continuación', 'votamos', 'telemático', 'republicanaizquierda', 'unidainiciativa', 'puede', 'bien', 'propuesta' ,'abstencion', 'mayoría', 'absoluta', 'pregunta', 'don', 'vamos', 'votar', 'llevar', 'cabo', 'millón', 'muchas', 'gracias'])

In [ ]:
# Se incorporan los nombres de los oradores como stopwords
from collections import Counter
nombres=[]
for i in base.namey.unique().tolist(): nombres.extend(str(i).split(' '))
nombres = [x.strip(' ') for x in nombres]
nombres = [x.strip(' †') for x in nombres]
nombres = [x.strip(',') for x in nombres]
nombres=list(Counter(nombres))
nombres.remove('')

In [ ]:
stopwords.extend(nombres)

In [ ]:
partidos=pd.read_csv('/content/drive/MyDrive/partidos.csv', encoding='latin1', sep=';')

In [ ]:
stopwords.extend(partidos['nombre'].unique())
stopwords.extend(partidos['1'].unique())
stopwords.extend(partidos['2'].unique())
stopwords.extend(partidos['3'].unique())

In [68]:
len(stopwords)

2483

# TF-IDF

In [ ]:
# El hiperparámetro min_df del TF-IDF elimina las palabras con menor frecuencia. Por ejemplo, las palabras que aparecen en menos que el 75% de los discursos. 
# A continuación se adopta un enfoque distinto: Se genera un vocabulario con todos los bigramas, y luego se elimina el 75% de los bigramas con menor TF-IDF. 
# Es decir que entre los 8 millones de bigramas se seleccionan los 2 millones de mayor TF-IDF. 

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range=[2,2], stop_words=stopwords)
bow = vect.fit_transform(base['tokens'])
total_features = len(vect.vocabulary_)

In [ ]:
total_features

8228144

In [90]:
sum_words = bow.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1])
vocabulary, _ = zip(*words_freq[:int(total_features * 0.975)])
less_vocabulary = list(vocabulary)

In [91]:
voc=vect.vocabulary_.keys()-less_vocabulary

In [92]:
len(voc)

205704

In [93]:
cv=TfidfVectorizer(ngram_range=[2,2], vocabulary=voc , stop_words=stopwords)
vec = cv.fit(base['tokens'])

In [94]:
matrix = vec.transform(base['tokens'])

In [95]:
matrix.shape

(334421, 205704)

# Número óptimo de clusters

In [ ]:
# LDA en Sklearn tiene un resultado de score correspondiente al log-likelihood. En base a ese criterio se identifica el número óptimo de clusters. 

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

search_params = {'n_components': [20, 40, 60, 80]}
model = LatentDirichletAllocation(max_iter=500, learning_method='online', learning_offset=50.,random_state=0, cv=2)
gridsearch = GridSearchCV(model,
                          param_grid=search_params,
                          n_jobs=-1,
                          verbose=1)
gridsearch.fit(matrix)
best_lda = gridsearch.best_estimator_

In [ ]:
cv_results_df = pd.DataFrame(gridsearch.cv_results_)
cv_results_df

In [ ]:
import seaborn as sns
sns.set(rc={"figure.dpi":150, 'savefig.dpi':150})
sns.pointplot(x="param_n_components",
              y="mean_test_score",
              data=cv_results_df)

# LDA con 10 clusters

In [96]:
from sklearn.decomposition import LatentDirichletAllocation

In [97]:
# Acá hay un problema porque se sigue usando 10 clusters cuando en realidad habría que chequear el número óptimo de clusters una vez definida la cantidad de bigramas a utilizar. 
# Sin embargo, Text as data dice que el número de clústers es generalmente arbitrario, y recomienda arrancar probando con 10. 
lda = LatentDirichletAllocation(n_components=10, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)     

In [98]:
lda.fit(matrix)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=5, random_state=0)

In [99]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx + 1))
        print(" , ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [100]:
display_topics(lda, feature_names=cv.get_feature_names_out(), no_top_words=20)

Topic 1:
toma consideración , dictamen sección , celebración extraordinaria , acuerd celebración , consideración proposición , suspend mañana , tiempo concluido , diputación permanent , sánchez robl , asunto exterior , texto dictamen , mañana hora , suficient miembro , permanent pertenecient , defensa mixto , senado proyecto , proced votacion , socialista acuerd , aprobada proposición , proyecto orgánica
Topic 2:
unido podemosen , confeder unido , aprobada resto , rechazado número , aprobada número , aprobada disposición , disposición adicion , disposición final , aprobada sección , vicepresidenta adelant , sección relativo , dictamen comisión , aprobada transaccion , aprobado artículo , ocupen escaño , texto dictamen , artículo único , tramitará proyecto , artículo apartado , resto mixto
Topic 3:
convergència unió , popular congreso , catalán convergència , izquierda unida , coalición canaria , congreso formula , poder judici , feder izquierda , guardia civil , sáenz santamaría , cana